In [1]:
import pandas as pd
import logging
import csv
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import udf
import re
import time
from pyspark.sql import functions as F
from pyspark.sql.functions import length,col, explode, upper, to_date, date_sub, lag, coalesce, lit, array_sort, when, arrays_zip, size, date_format, explode_outer, from_json, concat, expr, array
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.utils import AnalysisException
from operator import itemgetter
import datetime, re, requests
from pyspark.sql import Window
from pyspark.sql.functions import concat, col, lit 
from pyspark.sql.functions import split
from pyspark.sql.functions import when
from pyspark.sql.functions import current_timestamp

In [8]:
# # read data from S3 to DataFrame
# # input: S3
# # output df['ROW_ID', 'BOS_FILE_EXTRACT', 'file_path', 'COLUMN_DEF']
# def read_s3_to_df(day_path):
#
#     # parameter
#     i = 0 # flag for initialize dataframe
#
#     # boto3 client
#     s3 = boto3.client('s3')
#     conn = client('s3')  # again assumes boto.cfg setup, assume AWS S3
#
#     # read data file by file
#     for key in conn.list_objects(Bucket='bos-etl')['Contents']:
#         path_key = key['Key']
#         if path_key.endswith('cleansed'):
#             if day_path in path_key:
#                 file = s3.get_object(Bucket='bos-etl', Key=path_key)
#                 txt = (file['Body'].read().decode('latin1'))
#                 #st_re = txt.replace("", ",")
#                 st_re_newline = txt.replace("!!! EOS !!!", "\n")
#                 st_re_split = st_re_newline.split("\n")
#                 df = pd.DataFrame(st_re_split)
#                 df.index.name = 'ROW_ID'
#                 df.rename({0:'BOS_FILE_EXTRACT'},axis='columns',inplace=True)
#                 df["COLUMN_DEF"]=df['BOS_FILE_EXTRACT'].replace(regex=r"\.*",value="")
#                 rslt_dfRFT_temp = df[df['COLUMN_DEF'] =='PAX']
#                 # rslt_dfRFT_temp = df
#                 if ~rslt_dfRFT_temp.empty:
#                     # print(path_key)
#                     rslt_dfRFT_temp.insert(0,'file_path', path_key)
#                     if (i ==0):
#                         rslt_dfRFT = rslt_dfRFT_temp
#                         i = 1
#                     else:
#                         rslt_dfRFT = pd.concat([rslt_dfRFT,rslt_dfRFT_temp])
#     return rslt_dfRFT
#
#
# # read data as DataFrame
# #select day
# day_path = 'date=2023-06-17'
# # select current system day
# # day_path = time.strftime('%Y-%m-%d')
# rslt_dfRFT = read_s3_to_df(day_path)
#
# # test
# #filter content
# # check_list=['123','456']
#
# # for i in range():
# #       rslt_dfRFT['BOS_FILE_EXTRACT'].filter(like=check_list[i])

In [14]:
# #test write to s3 csv
# from io import StringIO
# bucket = 'bos-etl' # already created on S3
# csv_buffer = StringIO()
# rslt_dfRFT.to_csv(csv_buffer)
# s3_resource = boto3.resource('s3')
# s3_resource.Object(bucket, 'write_back/df.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'E5XW3WFJZ77P4QQF', 'HostId': 'En2Wle7HGBj5tEq9b5hiR5QHRvYocV4ymHKqbazUpsJeUXjqoGfoKLTM/g8I+NQfDExdkFDbk00=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'En2Wle7HGBj5tEq9b5hiR5QHRvYocV4ymHKqbazUpsJeUXjqoGfoKLTM/g8I+NQfDExdkFDbk00=', 'x-amz-request-id': 'E5XW3WFJZ77P4QQF', 'date': 'Wed, 26 Jul 2023 19:41:06 GMT', 'x-amz-version-id': 'aOLY0w2HKoTQf5j3Ee_OxQwCwG8vGbmp', 'x-amz-server-side-encryption': 'AES256', 'etag': '"e2b6a925db5723e142358c7a5fc0129d"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 0}, 'ETag': '"e2b6a925db5723e142358c7a5fc0129d"', 'ServerSideEncryption': 'AES256', 'VersionId': 'aOLY0w2HKoTQf5j3Ee_OxQwCwG8vGbmp'}


In [2]:
# read csv
rslt_dfRFT = pd.read_csv("source/input/1_df_RFT_PAT_PAX.csv")

In [3]:
# import sys
# from awsglue.transforms import *
# from awsglue.utils import getResolvedOptions
# from pyspark.context import SparkContext
# from awsglue.context import GlueContext
# from awsglue.job import Job
#
# ## @params: [JOB_NAME]
# args = getResolvedOptions(sys.argv, ['JOB_NAME'])
#
# sc = SparkContext()
# glueContext = GlueContext(sc)
# spark = glueContext.spark_session
# job = Job(glueContext)
# job.init(args['JOB_NAME'], args)

In [4]:
# Pyspark Dataframe
from pyspark.sql import SparkSession
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()
#Create PySpark DataFrame from Pandas
sparkDF=spark.createDataFrame(rslt_dfRFT)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/26 21:39:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# Split Raw Data
# input: spark dataframe
# output: ['split','ITI', 'FAR','FOP_AND_FAR2','END','CER','EXC','split2','DCI','SAL','EXS','split3','split4']

def split_raw_data(sparkDF):
    data = sparkDF.withColumn("split", F.split("BOS_FILE_EXTRACT", "<")).withColumn("ITI", F.element_at("split", 2)).withColumn("FAR", F.element_at("split", 3))\
    .withColumn("FOP_AND_FAR2", F.element_at("split", 4)).withColumn("END", F.element_at("split", 5)).withColumn("CER", F.element_at("split", 6))\
    .withColumn("EXC", F.element_at("split", 7))\
    .withColumn("split2", F.split("BOS_FILE_EXTRACT", "<")).withColumn("DCI", F.element_at("split2", 1)).withColumn("SAL", F.element_at("split2", 2))\
    .withColumn("EXS", F.element_at("split2", -1))\
    .withColumn("split3", F.split("BOS_FILE_EXTRACT", "<")).withColumn("TAX2", F.element_at("split3", 2))\
    .withColumn("split4", F.split("BOS_FILE_EXTRACT", "<")).withColumn("TAX1", F.element_at("split4", 2))\
    .withColumn("SAL2", F.element_at("split2", 3))
    return data

df1 = split_raw_data(sparkDF)

In [6]:
df1.head(1)

[Row(ROW_ID=45, file_path='data/date=2023-06-17/DCALLRECORDSAM.pgp.asc_3e5155823ea22a963da362086c693cf3_cleansed', BOS_FILE_EXTRACT='RFT\x8022521623\x8020230618\x8023061805093326\x80125\x802161454822\x804\x8017JUN23\x80\x80IAR\x80BA/D/ALLRECORDS/18JUN23\x80103\x80<\x80\x80\x80\x80\x80953.65\x800.00\x800.00\x80358.00\x80595.65\x800.00\x800.00\x800.00\x800.00\x80\x80\x80\x81125\x802161454822\x801234\x80<\x82\x80REDACTED REF NAME\x80F\x80-953.65\x800.00\x800.00\x80\x80-953.65\x80CA\x80CASH\x80<\x83', COLUMN_DEF='RFT', split=['RFT\x8022521623\x8020230618\x8023061805093326\x80125\x802161454822\x804\x8017JUN23\x80\x80IAR\x80BA/D/ALLRECORDS/18JUN23\x80103\x80<\x80\x80\x80\x80\x80953.65\x800.00\x800.00\x80358.00\x80595.65\x800.00\x800.00\x800.00\x800.00\x80\x80\x80\x81125\x802161454822\x801234\x80<\x82\x80REDACTED REF NAME\x80F\x80-953.65\x800.00\x800.00\x80\x80-953.65\x80CA\x80CASH\x80<\x83'], ITI=None, FAR=None, FOP_AND_FAR2=None, END=None, CER=None, EXC=None, split2=['RFT\x8022521623\x80202

In [65]:
split_df = df1.select("COLUMN_DEF","DCI","SAL","TAX1","TAX2","ITI","FAR","FOP_AND_FAR2","END","CER","EXC","EXS",split(df1.TAX1, '<').alias('split_text'),split(df1.FOP_AND_FAR2,'N<').alias("Splittext3"),"file_path","SAL2")
split_df_type=split_df.selectExpr("column_def","DCI","SAL","concat(split_text[0],',',TAX2) as TAX","ITI","FAR", "Case when column_def=='PAT' then FOP_AND_FAR2 else Splittext3[1] end as FOP_T","Case when column_def=='PAT' then CER else END end as END_T","Case when column_def=='PAT' then EXC else CER end as CER_T","Case when column_def=='PAT' then '' else EXC end as EXC_T","Case when column_def=='PAT' then '' else CER end as EXS_T","file_path","SAL2")

In [66]:
split_df.head(1)

[Row(COLUMN_DEF='RFT', DCI='RFT\x8022521623\x8020230618\x8023061805093326\x80125\x802161454822\x804\x8017JUN23\x80\x80IAR\x80BA/D/ALLRECORDS/18JUN23\x80103\x80', SAL='\x80\x80\x80\x80\x80953.65\x800.00\x800.00\x80358.00\x80595.65\x800.00\x800.00\x800.00\x800.00\x80\x80\x80\x81125\x802161454822\x801234\x80', TAX1=None, TAX2='\x80REDACTED REF NAME\x80F\x80-953.65\x800.00\x800.00\x80\x80-953.65\x80CA\x80CASH\x80<\x83', ITI=None, FAR=None, FOP_AND_FAR2=None, END=None, CER=None, EXC=None, EXS='\x83', split_text=None, Splittext3=None, file_path='data/date=2023-06-17/DCALLRECORDSAM.pgp.asc_3e5155823ea22a963da362086c693cf3_cleansed', SAL2='\x82\x80REDACTED REF NAME\x80F\x80-953.65\x800.00\x800.00\x80\x80-953.65\x80CA\x80CASH\x80')]

In [67]:
# # df2.write.options
# split_df_type.coalesce(1).write.format('csv').option('header','true').mode("overwrite").save("s3://bos-etl/write_back/")

In [112]:
# FTS - 2610  Commision: "CAD" - > "USD"
# input: Dataframe
# output: ['Transaction_Date','Source','Booking_channel','DCI_split','DCI','Agency_code',]

def split_col(split_df_type):
    data=split_df_type.withColumn("Transaction_Date",current_timestamp()).withColumn("Source",F.lit('BOS')).withColumn("Booking_channel",F.lit('WEB')).withColumn("DCI_split", F.split("DCI", "")).withColumn("Agency_code", F.element_at("DCI_split", 2)).withColumn("Ticket_No", concat(F.element_at("DCI_split", 5),F.element_at("DCI_split", 6))).withColumn("Issue_Date", F.to_date(F.element_at("DCI_split", 8),"ddMMMyy"))\
    .withColumn("SAL_split", F.split("SAL", "")).withColumn("PNR", F.element_at("SAL_split", 15)).withColumn("Tour_code", F.regexp_replace(F.element_at("SAL_split", 17),'','')).withColumn("Passenger", F.element_at("SAL_split", 16)).withColumn("Coupon_used", F.element_at("SAL_split", 7)).withColumn("Original_Fare", F.element_at("SAL_split", 9)).withColumn("Fare", F.element_at("SAL_split", 11)).withColumn("Org_currency", F.element_at("SAL_split", 10)).withColumn("Tax_Amt", F.element_at("SAL_split", 12)).withColumn("Total_amt", F.element_at("SAL_split", 8))\
    .withColumn("ITI_split", F.split("ITI", "<"))\
    .withColumn("Leg2", F.element_at("ITI_split", 13)).withColumn("Leg3", F.element_at("ITI_split", 22)).withColumn("Leg4", F.element_at("ITI_split", 31))\
    .withColumn("FOP_split", F.split("FOP_T", ""))\
    .withColumn("TAX_split", F.split("tax", "<"))\
    .withColumn("Commission", concat(F.lit('[{"type":"BASE","amount":'),F.element_at("SAL_split", 13),f.lit(',"currency":"USD","commissionRate":'),F.element_at("SAL_split", 14),f.lit('}]')))\
    .withColumn("Fare_split", F.split("FAR", "<")).withColumn("FOP_split", F.split("FOP_T", "<")).withColumn("SAL2_split", F.split("SAL2", "")).withColumn("Passenger_rfnd", F.element_at("SAL2_split", 2))\
    .withColumn("EXC_split", F.split("EXC_T", ""))
    
    return data

bos_df = split_col(split_df_type)

In [113]:
bos_df.head(1)

[Row(column_def='RFT', DCI='RFT\x8022521623\x8020230618\x8023061805093326\x80125\x802161454822\x804\x8017JUN23\x80\x80IAR\x80BA/D/ALLRECORDS/18JUN23\x80103\x80', SAL='\x80\x80\x80\x80\x80953.65\x800.00\x800.00\x80358.00\x80595.65\x800.00\x800.00\x800.00\x800.00\x80\x80\x80\x81125\x802161454822\x801234\x80', TAX=None, ITI=None, FAR=None, FOP_T=None, END_T=None, CER_T=None, EXC_T=None, EXS_T=None, file_path='data/date=2023-06-17/DCALLRECORDSAM.pgp.asc_3e5155823ea22a963da362086c693cf3_cleansed', SAL2='\x82\x80REDACTED REF NAME\x80F\x80-953.65\x800.00\x800.00\x80\x80-953.65\x80CA\x80CASH\x80', Transaction_Date=datetime.datetime(2023, 7, 26, 23, 6, 12, 334267), Source='BOS', Booking_channel='WEB', DCI_split=['RFT', '22521623', '20230618', '23061805093326', '125', '2161454822', '4', '17JUN23', '', 'IAR', 'BA/D/ALLRECORDS/18JUN23', '103', ''], Agency_code='22521623', Ticket_No='1252161454822', Issue_Date=datetime.date(2023, 6, 17), SAL_split=['', '', '', '', '', '953.65', '0.00', '0.00', '358

In [114]:
### FTS - 2607
bos_df_csv=bos_df.selectExpr("ITI_split","column_def","Transaction_Date","Agency_code","'06.07_DCALLRECORDSAM' as Filename","Source","Booking_channel","case when column_def=='PAX' then 'EXCH-TKTT' when column_def=='PAT' then 'TKTT' else 'RFND' end as ticket_type", "1 as version_no","Ticket_No","Issue_date","PNR","Tour_code","case when column_def=='RFT' then passenger_rfnd else Passenger end as Passenger_name","1 as Passenger_Count","Coupon_used","'USA' AS Country_code","Original_Fare","case when Fare==0 then original_fare else fare end as Fare_amt","Org_currency","round((case when Fare==0 then original_fare else fare end)/Original_Fare,3) as exch_rate","case when Fare==0 then original_fare else fare end as comm_amt","Tax_Amt","Total_amt","'USD' as curr_code","ticket_no as org_ticket_no","Commission","TAX_split","FOP_split","Fare_split","file_path","EXC_split")

In [124]:
bos_df_csv.select('ticket_type','PNR','Passenger_name',"Coupon_used","org_ticket_no").show()

+-----------+---------+-----------------+-----------+-------------+
|ticket_type|      PNR|   Passenger_name|Coupon_used|org_ticket_no|
+-----------+---------+-----------------+-----------+-------------+
|       RFND|         |REDACTED REF NAME|       0.00|1252161454822|
|  EXCH-TKTT|HNKNVK/AA|REDACTED SAL NAME|       FFVV|0017976703339|
|       TKTT|   UOAGUL|REDACTED SAL NAME|       FFVV|0742100377105|
+-----------+---------+-----------------+-----------+-------------+



In [116]:
# FTS - 2609
def fc_Fare(li, column_def, org_ticket_no):
    fare_v = ''
    fare = ''
    fare_end_v = ''
    if column_def != 'RFT':
        # print(row["Fare_split"])

        # li = row["Fare_split"]
        # print(li)
        fare_v = ''
        # print (fare_v)
        if li is not None:
            # FTS - 2609
            # fare_end_v = []
            fare_end_v = ''
            fare_v = '['
            i = 0
            # print (fare_v)
            for ii in li:
                i = i + 1
                ii_split = ii.split("")

                if i > 1:
                    fare_v = fare_v + ',{"sequence":' + str(i) + ',"content":"' + ii_split[0] + '"}'
                    fare = fare + ',' + ii_split[0]
                else:
                    fare_v = fare_v + '{"sequence":' + str(i) + ',"content":"' + ii_split[0] + '"}'
                    fare = ii_split[0]
                # print (fare_v)
            fare_v = fare_v + ']'
            # print(fare_v)
        # FTS - 2609 spilt by "END" + " "
        # fare_split = fare.split(" ")
        fare_split = fare.split("END" + " ")[0].split(" ")
        # print(fare_split)

        for element in fare_split:
            # FTS - 2609 add ~(element.startswith("Q") & element[1].isdigit())
            # if element.endswith("END"):
            if len(element) > 1:
                if (False if (element.startswith("Q") & element[1].isdigit()) else True):
                    # FTS - 2609 list -> str
                    fare_end_v = fare_end_v + element

    return [fare_v, fare, fare_end_v]

fc_Fare = udf(fc_Fare, ArrayType(StringType()))
fc_Fare_list = fc_Fare('Fare_split', 'column_def')
bos_df_csv = bos_df_csv.withColumn('Fare_Cons', fc_Fare_list[0])\
    .withColumn("Fare", fc_Fare_list[1]).withColumn("Fare_end",fc_Fare_list[2]).withColumn("Fare_split", F.split("Fare", " "))

In [117]:
bos_df_csv.select('Fare_Cons', "Fare_end", "Fare_split").show()

+--------------------+--------------------+--------------------+
|           Fare_Cons|            Fare_end|          Fare_split|
+--------------------+--------------------+--------------------+
|                    |                    |                  []|
|[{"sequence":1,"c...|ORFAAX/WASAAGRR16...|[ORF, AA, X/WAS, ...|
|[{"sequence":1,"c...|AMSKLMAD119.12KLA...|[AMS, KL, MAD119....|
+--------------------+--------------------+--------------------+



In [118]:
# FTS - 2607
# tax
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
def fc(li, column_def):
    tax_v = ''
    if column_def != 'RFT':
        tax_v =''
        if li is not None:
            #print(li)
            tax_v ='['
            i=0
            for ii in li:
                ii_split = ii.split(",")
                # FTS 2607 Bug 2 & 3 : filter string which not start with digit
                ii_split = [x for x in ii_split if x[0].isdigit()]
                #print(ii_split)
                for i1 in ii_split:
                    i=i+1
                    i1_split = i1.split("")
                    #print(i1_split)
                    if i>1:
                        # FTS-2607 Bug 1
                        tax_v = tax_v+ ',{"type":"'+ i1_split[1] +'","amount":'+i1_split[0] +',"currency":"' +  "Org_currency" + '"}'
                    else:
                        # FTS-2607 Bug 1
                        tax_v = tax_v+ '{"type":"'+ i1_split[1] +'","amount":'+i1_split[0] +',"currency":"' +   "Org_currency" + '"}'
            tax_v=tax_v+']'
    return tax_v

fc = udf(fc, StringType())
bos_df_csv = bos_df_csv.withColumn('Tax_v', fc('TAX_split', 'column_def'))
bos_df_csv=bos_df_csv.selectExpr("ITI_split","Fare_end","column_def","Transaction_Date","Agency_code","Filename","Source","Booking_channel","ticket_type", "version_no","Ticket_No","Issue_date","PNR","Tour_code","Passenger_name","Passenger_Count","Coupon_used","Country_code","Original_Fare","Fare_amt","Org_currency","exch_rate","comm_amt","Tax_Amt","Total_amt","curr_code","org_ticket_no","Commission","Fare_Cons","case when column_def !='RFT' then replace(Tax_v,'Org_currency',Org_currency) end as Tax",'FOP_split',"file_path","EXC_split")

In [119]:
bos_df_csv.select('column_def','Tax','EXC_split').show()

+----------+--------------------+--------------------+
|column_def|                 Tax|           EXC_split|
+----------+--------------------+--------------------+
|       RFT|                null|                null|
|       PAX|[{"type":"US","am...|[001, 7976703339,...|
|       PAT|[{"type":"YR","am...|                  []|
+----------+--------------------+--------------------+



In [130]:
bos_df_csv2=bos_df_csv

In [131]:
# FTS - 2606
# payment
def fc_FOP(li, column_def,EXC_split, org_ticket_no):
    print(column_def)
    fop_v =''
    if column_def != 'RFT':
        # li = row["FOP_split"]
        if li is not None:
            #print(li)
            fop_v ='['
            i=0
            for ii in li:
                ii_split = ii.split(",")
                #print(ii_split)
                for i1 in ii_split:
                    i=i+1
                    i1_split = i1.split("") #FOP
                    #print(i1_split)
                    mode = i1_split[0][:2] if column_def == 'PAT' else 'EX'
                    type1 = i1_split[0] if column_def == 'PAT' else 'EX'
                    amount = i1_split[5] if column_def == 'PAT' else '0.00'
                    accountNumber = i1_split[1] if column_def == 'PAT' else \
                        EXC_split[0] + EXC_split[1] + EXC_split[2] + EXC_split[18]
                    org_ticket_no = org_ticket_no if column_def == 'PAT' else 'EX' + accountNumber[0:13]
                    print(accountNumber)
                    if i>1 :
                        # FTS - 2606 "mode" add ",  mode <-> type
                        fop_v = fop_v+ ',{"mode":"'+ mode +'","type":"'+ type1 + '","amount":'+ amount +',"accountNumber":"'+accountNumber+ '","approvalCode":"'+ i1_split[4]+'","invoiceNumber":"","invoiceNumber":"","currency":"'+"Org_currency" + '"}'
                    else:
                        # FTS - 2606 "mode" add ",  mode <-> type
                        fop_v = fop_v+ '{"mode":"'+ mode +'","type":"'+ type1 +'","amount":'+ amount +',"accountNumber":"'+accountNumber+ '","approvalCode":"'+i1_split[4]+'","invoiceNumber":"","invoiceNumber":"","currency":"'+"Org_currency" + '"}'
            fop_v=fop_v+']'

    return [fop_v, org_ticket_no]

fc_FOP = udf(fc_FOP, ArrayType(StringType()))
fc_FOP_list = fc_FOP('FOP_split','column_def','EXC_split','org_ticket_no')
bos_df_csv2 = bos_df_csv2.withColumn('Fop_v', fc_FOP_list[0]).withColumn('org_ticket_no', fc_FOP_list[1])
bos_df_csv2=bos_df_csv2.selectExpr("ITI_split","Fare_end","column_def","Transaction_Date","Agency_code","Filename","Source","Booking_channel","ticket_type", "version_no","Ticket_No","Issue_date","PNR","Tour_code","Passenger_name","Passenger_Count","Coupon_used","Country_code","Original_Fare","Fare_amt","Org_currency","exch_rate","comm_amt","Tax_Amt","Total_amt","curr_code","org_ticket_no","Commission","Fare_Cons","Tax","case when column_def !='RFT' then replace(Fop_v,'Org_currency',Org_currency) end as FOP","file_path")

In [132]:
bos_df_csv2.select('FOP','Commission','org_ticket_no').head(3)

PATPAX
00179767033390REDACTED EXC NAME

CASH
RFT


[Row(FOP=None, Commission='[{"type":"BASE","amount":0.00,"currency":"USD","commissionRate":0.00}]', org_ticket_no='1252161454822'),
 Row(FOP='[{"mode":"EX","type":"EX","amount":0.00,"accountNumber":"00179767033390REDACTED EXC NAME","approvalCode":"","invoiceNumber":"","invoiceNumber":"","currency":"USD"}]', Commission='[{"type":"BASE","amount":1.00,"currency":"USD","commissionRate":0.60}]', org_ticket_no='EX0017976703339'),
 Row(FOP='[{"mode":"CA","type":"CA","amount":302.20,"accountNumber":"CASH","approvalCode":"","invoiceNumber":"","invoiceNumber":"","currency":"USD"}]', Commission='[{"type":"BASE","amount":0.00,"currency":"USD","commissionRate":0.00}]', org_ticket_no='0742100377105')]

In [17]:
from datetime import datetime
# FTS - 2608 rewrtie function
# FTS - 2608 "currency": "CAD" -> "currency": "USD"
def fc_LEG(li, fare_row, ticket_no, issue_date, column_def):
    leg_v =''
    if column_def != 'RFT':
        #print(row)
        # li = row["ITI_split"]
        # fare_row = row["Fare_end"]
        issue_year = issue_date.year
        issue_date = issue_date.strftime('%Y-%m-%d')
        leap_year = '2024' # not use this value just avoid error from strptime
        if li is not None:
            #print(li)
            leg_v ='[' # segment 1
            leg_v_1 = '[' #segment 2
            # i=0
            # FTS - 2608
            i = 0
            j = 1 # index of fare , max len 2
            fareval = []
            if (len(li) > 1): # if two segment, reverse, direction right to left
                li.reverse()
                i = 3
            for ii in li:
                ii_split = ii.split(",")
                #print(ii_split)
                for i1 in ii_split:
                    fareval = re.findall(r'\d+.\d+', str(fare_row))
                    if len(fareval)==0:
                        total_fareval = '0.00'
                        fareval=['0.00','0.00','0.00','0.00']
                    else:
                        total_fareval = fareval[-1]
                        fareval = fareval[:-1]
                    #print(fareval)
                    i = i - 1 # reverse
                    i1_split = i1.split("")
                    if len(i1_split[0])!=0:
                        conj= ticket_no[0:3] + i1_split[0]
                    else:
                        conj='CONJ'

                    if i>1:
                       if len(i1_split[30])!=0:
                            if len(i1_split[35])!=0:
                                dep_day = datetime.strptime(i1_split[35] + leap_year,'%d%b%Y').strftime("%m-%d")
                                dep_year = str(issue_year) if dep_day >= issue_date[5:] else str(int(issue_year)    + 1)
                                dep_on = pd.datetime.strptime(i1_split[35]+ dep_year,'%d%b%Y')
                            else:
                                dep_on= 'NULL'
                            if i1_split[29]=='X':
                                fare = str('0.00')
                            else:
                                if (len(fareval) == 0):
                                    fare ='NULL'
                                elif (j <= len(fareval)):
                                    fare = fareval[-j]
                                    j = j + 1
                                else:
                                    fare = 'NULL'
                                    j = j + 1
                            leg_v_1 = ',{"departure":"'+ i1_split[30] +'","destination":"'+i1_split[31]     +'","seatClass":"' +  i1_split[34] +'","conjunction":"'+  conj  +'","carrier":"'+i1_split[32]+'","tripCode":"'+i1_split[33]+'","departureOn":"'+str( dep_on)+'","designator":"'+i1_split[37]+'","stopOver":"'+i1_split[   29]+'","flyerCode":"","fare":'+ fare + ',"currency": "USD","originalFare":'+ fare  +',"originalCurrency":"'+'org_currency'+'"}'

                       if len(i1_split[21])!=0:
                            if len(i1_split[26])!=0:
                                dep_day = datetime.strptime(i1_split[26] + leap_year,'%d%b%Y').strftime("%m-%d")
                                dep_year = str(issue_year) if dep_day >= issue_date[5:] else str(int(issue_year)    + 1)
                                dep_on = pd.datetime.strptime(i1_split[26]+ dep_year,'%d%b%Y')
                            else:
                                dep_on= 'NULL'
                            if i1_split[20]=='X':
                                fare = str('0.00')
                            else:
                                if (len(fareval) == 0):
                                    fare ='NULL'
                                elif (j <= len(fareval)):
                                    fare = fareval[-j]
                                    j = j + 1
                                else:
                                    fare = 'NULL'
                                    j = j + 1
                            leg_v_1 = '' if leg_v_1 == '[' else leg_v_1
                            leg_v_1 = ',{"departure":"'+ i1_split[21] +'","destination":"'+i1_split[22]     +'","seatClass":"' + i1_split[25] +'","conjunction":"'+  conj   +'","carrier":"'+i1_split[23]+'","tripCode":"'+i1_split[24]+'","departureOn":"'+str(  dep_on)+'","designator":"'+i1_split[28]+'","stopOver":"'+i1_split[    20]+'","flyerCode":"","fare":'+ fare +',"currency": "USD","originalFare":'+ fare    +',"originalCurrency":"'+'org_currency'+'"}' + leg_v_1

                       if len(i1_split[12])!=0:
                            if len(i1_split[17])!=0:
                                dep_day = datetime.strptime(i1_split[17] + leap_year,'%d%b%Y').strftime("%m-%d")
                                dep_year = str(issue_year) if dep_day >= issue_date[5:] else str(int(issue_year)    + 1)
                                dep_on = pd.datetime.strptime(i1_split[17]+ dep_year,'%d%b%Y')
                            else:
                                dep_on= 'NULL'
                            if i1_split[11]=='X':
                                fare = str('0.00')
                            else:
                                if (len(fareval) == 0):
                                    fare ='NULL'
                                elif (j <= len(fareval)):
                                    fare = fareval[-j]
                                    j = j + 1
                                else:
                                    fare = 'NULL'
                                    j = j + 1
                            leg_v_1 = '' if leg_v_1 == '[' else leg_v_1
                            leg_v_1 = ',{"departure":"'+ i1_split[12] +'","destination":"'+i1_split[13]     +'","seatClass":"' + i1_split[16] +'","conjunction":"'+  conj   +'","carrier":"'+i1_split[14]+'","tripCode":"'+i1_split[15]+'","departureOn":"'+str(  dep_on)+'","designator":"'+i1_split[19]+'","stopOver":"'+i1_split[    11]+'","flyerCode":"","fare":'+ fare +',"currency": "USD","originalFare":'+ fare    +',"originalCurrency":"'+'org_currency'+'"}' + leg_v_1

                       if len(i1_split[3])!=0:
                            if len(i1_split[8])!=0:
                                dep_day = datetime.strptime(i1_split[8] + leap_year,'%d%b%Y').strftime("%m-%d")
                                dep_year = str(issue_year) if dep_day >= issue_date[5:] else str(int(issue_year)    + 1)
                                dep_on = pd.datetime.strptime(i1_split[8]+ dep_year,'%d%b%Y')
                            else:
                                dep_on= 'NULL'
                            if i1_split[2]=='X':
                                fare = str('0.00')
                            else:
                                if (len(fareval) == 0):
                                    fare ='NULL'
                                elif (j <= len(fareval)):
                                    fare = fareval[-j]
                                    j = j + 1
                                else:
                                    fare = 'NULL'
                                    j = j + 1

                            leg_v_1 = '' if leg_v_1 == '[' else leg_v_1
                            leg_v_1 = ',{"departure":"'+ i1_split[3] +'","destination":"'+i1_split[4]   +'","seatClass":"' + i1_split[7] +'","conjunction":"'+  conj +'","carrier":"'+i1_split    [5]+'","tripCode":"'+i1_split[6]+'","departureOn":"'+str(   dep_on)+'","designator":"'+i1_split[10]+'","stopOver":"'+i1_split[ 2]+'","flyerCode":"","fare":'+ fare +',"currency": "USD","originalFare":'+ fare  +',"originalCurrency":"'+'org_currency'+'"}' + leg_v_1

                    else:
                        if len(i1_split[30])!=0:
                            if len(i1_split[35])!=0:
                                dep_day = datetime.strptime(i1_split[35] + leap_year,'%d%b%Y').strftime("%m-%d")
                                dep_year = str(issue_year) if dep_day >= issue_date[5:] else str(int(issue_year)    + 1)
                                dep_on = pd.datetime.strptime(i1_split[35]+ dep_year,'%d%b%Y')
                            else:
                                dep_on= 'NULL'
                            if i1_split[29]=='X':
                                fare = str('0.00')
                            else:
                                if (len(fareval) == 0):
                                    fare ='NULL'
                                elif (j <= len(fareval)):
                                    fare = fareval[-j]
                                    j = j + 1
                                else:
                                    fare = 'NULL'
                                    j = j + 1

                            leg_v = ',{"departure":"'+ i1_split[30] +'","destination":"'+i1_split[31]   +'","seatClass":"' + i1_split[34] +'","conjunction":"'+  conj     +'","carrier":"'+i1_split[32]+'","tripCode":"'+i1_split[33]+'","departureOn":"'+str(    dep_on)+'","designator":"'+i1_split[37]+'","stopOver":"'+i1_split[  29]+'","flyerCode":"","fare":'+ fare + ',"currency": "USD","originalFare":'+ fare     +',"originalCurrency":"'+'org_currency'+'"}'

                        if len(i1_split[21])!=0:
                            if len(i1_split[26])!=0:
                                dep_day = datetime.strptime(i1_split[26] + leap_year,'%d%b%Y').strftime("%m-%d")
                                dep_year = str(issue_year) if dep_day >= issue_date[5:] else str(int(issue_year)    + 1)
                                dep_on = pd.datetime.strptime(i1_split[26]+ dep_year,'%d%b%Y')
                            else:
                                dep_on= 'NULL'
                            if i1_split[20]=='X':
                                fare = str('0.00')
                            else:
                                if (len(fareval) == 0):
                                    fare ='NULL'
                                elif (j <= len(fareval)):
                                    fare = fareval[-j]
                                    j = j + 1
                                else:
                                    fare = 'NULL'
                                    j = j + 1
                            leg_v = '' if leg_v == '[' else leg_v
                            leg_v =  ',{"departure":"'+ i1_split[21] +'","destination":"'+i1_split[22]  +'","seatClass":"' + i1_split[25] +'","conjunction":"'+  conj    +'","carrier":"'+i1_split[23]+'","tripCode":"'+i1_split[24]+'","departureOn":"'+str(   dep_on)+'","designator":"'+i1_split[28]+'","stopOver":"'+i1_split[ 20]+'","flyerCode":"","fare":'+ fare +',"currency": "USD","originalFare":'+ fare     +',"originalCurrency":"'+'org_currency'+'"}' + leg_v

                        if len(i1_split[12])!=0:
                            if len(i1_split[17])!=0:
                                dep_day = datetime.strptime(i1_split[17] + leap_year,'%d%b%Y').strftime("%m-%d")
                                dep_year = str(issue_year) if dep_day >= issue_date[5:] else str(int(issue_year)    + 1)
                                dep_on = pd.datetime.strptime(i1_split[17]+ dep_year,'%d%b%Y')
                            else:
                                dep_on= 'NULL'
                            if i1_split[11]=='X':
                                fare = str('0.00')
                            else:
                                if (len(fareval) == 0):
                                    fare ='NULL'
                                elif (j <= len(fareval)):
                                    fare = fareval[-j]
                                    j = j + 1
                                else:
                                    fare = 'NULL'
                                    j = j + 1
                            leg_v = '' if leg_v == '[' else leg_v
                            leg_v =   ',{"departure":"'+ i1_split[12] +'","destination":"'+i1_split[13]     +'","seatClass":"' + i1_split[16] +'","conjunction":"'+  conj   +'","carrier":"'+i1_split[14]+'","tripCode":"'+i1_split[15]+'","departureOn":"'+str(  dep_on)+'","designator":"'+i1_split[19]+'","stopOver":"'+i1_split[    11]+'","flyerCode":"","fare":'+ fare +',"currency": "USD","originalFare":'+ fare    +',"originalCurrency":"'+'org_currency'+'"}' + leg_v

                        if len(i1_split[3])!=0:
                            #print(conj)
                            if len(i1_split[8])!=0:
                                dep_day = datetime.strptime(i1_split[8] + leap_year,'%d%b%Y').strftime("%m-%d")
                                dep_year = str(issue_year) if dep_day >= issue_date[5:] else str(int(issue_year)    + 1)
                                dep_on = pd.datetime.strptime(i1_split[8]+ dep_year,'%d%b%Y')
                            else:
                                dep_on= 'NULL'
                            if i1_split[2]=='X':
                                fare = str('0.00')
                            else:
                                if (len(fareval) == 0):
                                    fare ='NULL'
                                elif (j <= len(fareval)):
                                    fare = fareval[-j]
                                    j = j + 1
                                else:
                                    fare = 'NULL'
                                    j = j + 1
                            leg_v = '' if leg_v == '[' else leg_v
                            leg_v = '[{"departure":"'+ i1_split[3] +'","destination":"'+i1_split[4]     +'","seatClass":"' + i1_split[7] +'","conjunction":"'+  conj +'","carrier":"'+i1_split  [5]+'","tripCode":"'+i1_split[6]+'","departureOn":"'+str( dep_on)+'","designator":"'+i1_split[10]+'","stopOver":"'+i1_split[   2]+'","flyerCode":"","fare":'+ fare +',"currency": "USD","originalFare":'+ fare    +',"originalCurrency":"'+'org_currency'+'"}' + leg_v

                        # segment 1 + segment2
                        leg_v_1 = '' if leg_v_1 == '[' else leg_v_1
                        leg_v = leg_v + leg_v_1
                    #print(pd.datetime.strptime(i1_split[8]+ str(datetime.now().year),'%d%b%Y'))
            leg_v=leg_v+']'

            # attributes: len(leg) > len(fare)
            attributes = '' if j == len(fareval) + 1 else ' # ' + 'FareLegMismatch:  leg: ' + str(j-1) +  ' does    not match fare: ' + str(len(fareval)) + ' # '

            # atributes: total_fareval <> leg: fareval
            leg_fer_sum = '%.2f'%sum([float(x[7:]) for x in re.findall(r'\"fare\"\:\d+\.\d+', leg_v)])
            attributes = attributes if total_fareval == leg_fer_sum else '#' + 'FareAmountMismatch: Leg sum     amount: ' + str(leg_fer_sum) + ' does not match fare amount: ' + str(total_fareval)  + ' # '

    return [leg_v, attributes]

fc_LEG = udf(fc_LEG, ArrayType(StringType()))
fc_LEG_list = fc_LEG('ITI_split','Fare_end','Ticket_no','Issue_date','column_def')
bos_df_csv = bos_df_csv.withColumn('Leg_v1', fc_LEG_list[0]).withColumn('attributes', fc_LEG_list[1])
bos_df_csv=bos_df_csv.selectExpr("FOP","column_def","Transaction_Date","Agency_code","Filename","Source","Booking_channel","ticket_type", "version_no","Ticket_No","Issue_date","PNR","Tour_code","Passenger_name","Passenger_Count","Coupon_used","Country_code","Original_Fare","Fare_amt","Org_currency","exch_rate","comm_amt","Tax_Amt","Total_amt","curr_code","org_ticket_no","Commission","Fare_Cons","Tax","case when column_def !='RFT' then replace(replace(Leg_v1,'CONJ',Ticket_no),'org_currency',Org_currency) else '[]' end as Legs","attributes","file_path")

In [18]:
bos_df_csv.head(1)

23/07/26 21:21:28 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/07/26 21:21:32 ERROR Executor: Exception in task 0.0 in stage 9.0 (TID 17)   
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/var/folders/d9/871p0v1d2cn4c6y7bj_xghhh0000gn/T/ipykernel_2618/134353313.py", line 31, in fc_Fare
UnboundLocalError: local variable 'fare' referenced before assignment

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:561)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:94)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:514)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/var/folders/d9/871p0v1d2cn4c6y7bj_xghhh0000gn/T/ipykernel_2618/134353313.py", line 31, in fc_Fare
UnboundLocalError: local variable 'fare' referenced before assignment


In [15]:
# Data Validation
# def
def check_float(data, msg, header, length, decimal = 0):
    try:
        data_1 = data
        data = data if len(str(data).split('.')[0]) <= (length - decimal) else 'NULL'
        msg = msg if str(data) == str(data_1) else msg + ' # ' + header + 'OutOfRange: ' + str(data_1) + 'is out of range of (' +  str(length) + ',' + str(decimal) + ') # '
    except Exception as e:
        data = 'NULL'
        msg = msg + '# ERROR ' + str(e) + ' # '
    return data, msg


def fc_DV(attributes,original_fare, exchange_rate,fare_amount,tax_amount,total_amount):

    # attributes
    attributes = '' if attributes is None else attributes

    # original_fare
    # numeric(12, 5)
    original_fare, attributes = check_float(data = original_fare, msg = attributes , header = 'original_fare', length = 12, decimal = 5)

    # exchange_rate
    # numeric(12, 5)
    exchange_rate, attributes = check_float(data = exchange_rate, msg = attributes , header = 'exchange_rate', length = 12, decimal = 5)

    # fare_amount
    # numeric(12, 5)
    fare_amount, attributes = check_float(data = fare_amount, msg = attributes , header = 'fare_amount', length = 12, decimal = 5)

    # tax_amount
    # numeric(12, 5)
    tax_amount, attributes = check_float(data = tax_amount, msg = attributes , header = 'tax_amount', length = 12, decimal = 5)

    # total_amount
    # numeric(12, 5)
    total_amount, attributes = check_float(data = total_amount, msg = attributes , header = 'total_amount', length = 12, decimal = 5)

    return [attributes,original_fare,exchange_rate,fare_amount,tax_amount,total_amount]

fc_DV = udf(fc_DV, ArrayType(StringType()))
fc_DV_list = fc_DV('attributes', 'Original_Fare', 'exch_rate', 'Fare_amt', 'Tax_Amt', 'Total_amt')
bos_df_csv = bos_df_csv\
    .withColumn('attributes', fc_DV_list[0])\
    .withColumn('Original_Fare', fc_DV_list[1])\
    .withColumn('exch_rate', fc_DV_list[2])\
    .withColumn('Fare_amt', fc_DV_list[3])\
    .withColumn('Tax_Amt', fc_DV_list[4])\
    .withColumn('Total_amt', fc_DV_list[5])

In [16]:
def fc_DV_2(attributes, Agency_code,Source,ticket_type,Coupon_used):

    # attributes
    attributes = '' if attributes is None else attributes

    # Agency_code
    # numeric(10, 0)
    Agency_code, attributes = check_float(data = Agency_code, msg = attributes , header = 'Agency_code', length = 10, decimal = 0)

    # Source
    # numeric(10, 0)
    Source, attributes = check_float(data = Source, msg = attributes , header = 'Source', length = 10, decimal = 0)

    # ticket_type
    # numeric(10, 0)
    ticket_type, attributes = check_float(data = ticket_type, msg = attributes , header = 'ticket_type', length = 10, decimal = 0)


    # Coupon_used
    # numeric(10, 0)
    Coupon_used, attributes = check_float(data = Coupon_used, msg = attributes , header = 'Coupon_used', length = 10, decimal = 0)


    return [attributes,Agency_code,Source,ticket_type,Coupon_used]

fc_DV_2 = udf(fc_DV_2, ArrayType(StringType()))
fc_DV_2_list = fc_DV_2('attributes', 'Agency_code','Source','ticket_type','Coupon_used')
bos_df_csv = bos_df_csv\
    .withColumn('attributes', fc_DV_2_list[0])\
    .withColumn('Agency_code', fc_DV_2_list[1])\
    .withColumn('Source', fc_DV_2_list[2])\
    .withColumn('ticket_type', fc_DV_2_list[3])\
    .withColumn('Coupon_used', fc_DV_2_list[4])

In [17]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame

sc = SparkContext.getOrCreate();
glueContext = GlueContext(sc)
spark = glueContext.spark_session

my_conn_options = {
    "dbtable": "flextravel.fx_trans_file",
    "database": "fts_cp_uat",
    "url": "jdbc:postgresql://flextravel-uat-serverless-pg.chzjoncadzav.ca-central-1.rds.amazonaws.com:5432/fts_cp_uat",
    "customJdbcDriverS3Path":"s3://flex-data-uat-canda-central/DEV/postgresql-42.6.0.jar",
    "customJdbcDriverClassName":"org.postgresql.Driver",
    "user":"flexuatuser",
    "password":"flexnewyearpwd@2022"
}

my_conn_options1 = {
    "dbtable": "flextravel.general_info",
    "database": "fts_cp_uat",
    "url": "jdbc:postgresql://flextravel-uat-serverless-pg.chzjoncadzav.ca-central-1.rds.amazonaws.com:5432/fts_cp_uat",
    "customJdbcDriverS3Path":"s3://flex-data-uat-canda-central/DEV/postgresql-42.6.0.jar",
    "customJdbcDriverClassName":"org.postgresql.Driver",
    "user":"flexuatuser",
    "password":"flexnewyearpwd@2022"
}

my_conn_options2 = {
    "dbtable": "public.fx_trans_bre_interim_bos_sprint2_test",
    "database": "fts_cp_uat",
    "url": "jdbc:postgresql://flextravel-uat-serverless-pg.chzjoncadzav.ca-central-1.rds.amazonaws.com:5432/fts_cp_uat",
    "customJdbcDriverS3Path":"s3://flex-data-uat-canda-central/DEV/postgresql-42.6.0.jar",
    "customJdbcDriverClassName":"org.postgresql.Driver",
    "user":"flexuatuser",
    "password":"flexnewyearpwd@2022"
}

df = glueContext.create_dynamic_frame.from_options(
    connection_type="postgresql",
    connection_options=my_conn_options,
    transformation_ctx="df",
)

df_GI = glueContext.create_dynamic_frame.from_options(
    connection_type="postgresql",
    connection_options=my_conn_options1,
    transformation_ctx="df",
)
GI_df = df_GI.toDF().select("id","tids_code")  



bos_df_csv=  bos_df_csv.withColumn( "Orginal_currency",F.when(length(col("Org_currency"))>5,'').otherwise(bos_df_csv.Org_currency))


bos_df_final = bos_df_csv.select("ticket_type","org_ticket_no","version_no","Filename","Transaction_Date","Agency_code","Source","Booking_channel","Ticket_No","PNR","Issue_date","Tour_code","Passenger_name","Passenger_Count","Coupon_used","Country_code","Legs","Original_Fare","Orginal_currency","exch_rate","Fare_amt","Tax_Amt","Total_amt","curr_code","FOP","Tax","Commission","Fare_Cons","attributes","file_path")
                                                           
Bos_df_GI= bos_df_final.join(GI_df,
               bos_df_csv.Agency_code == GI_df.tids_code, 
               "left")  


Bos_df_GI=Bos_df_GI.withColumnRenamed("id","agent_id")

#Bos_df_GI.show(2, truncate=False)

# df1 = df.toDF().select("id", "supplier_id","sup_srv_map_id","file_name").where(df["file_name"] =='06.07_DCALLRECORDSAM')
# #df2=df1.withColumnRenamed("sup_srv_map_id","col0").withColumnRenamed("file_name","col1")
# # df1 = df.filter(f=lambda x: x["sup_srv_map_id"] in [65])

df1 = df.toDF()["id", "supplier_id","sup_srv_map_id","file_name"]
df1 = df1[df1["file_name"] =='06.07_DCALLRECORDSAM']


#df1.show()

Bos_df_file= Bos_df_GI.join(df1,
               bos_df_csv.Filename == df1.file_name, 
               "left")    

# Bos_df_file.show()

# FTS - 2598 ["Tax_Amt"].cast(IntegerType()) -> DoubleType, ["Total_amt"].cast(IntegerType()) -> DoubleType
Bos_df_file= Bos_df_file.withColumn("Original_Fare", Bos_df_file["Original_Fare"].cast(DoubleType())).withColumn("Fare_amt", Bos_df_file["Fare_amt"].cast(DoubleType())).withColumn("Tax_Amt", Bos_df_file["Tax_Amt"].cast(DoubleType()))\
            .withColumn("Total_amt", Bos_df_file["Total_amt"].cast(DoubleType())) .withColumn("exch_rate", Bos_df_file["exch_rate"].cast(DoubleType()))

Bos_df_file= Bos_df_file.withColumn("Commission_Amount", Bos_df_file["Fare_amt"].cast(DoubleType()))
                        
Bos_df_write = Bos_df_file.select("Transaction_Date","Agency_code","Source","Booking_channel","Ticket_No","PNR","Issue_date","Tour_code","Passenger_name","Passenger_Count","Coupon_used","Country_code","Legs","Original_Fare","Orginal_currency","exch_rate","Fare_amt","Tax_Amt","Total_amt","curr_code","FOP","Tax","Commission","Fare_Cons","agent_id","id","supplier_id","sup_srv_map_id","ticket_type","version_no","org_ticket_no","Commission_Amount","attributes","file_path")


Bos_df_write = Bos_df_write.withColumnRenamed("Transaction_Date","transaction_date").withColumnRenamed("Agency_code","agency_code").withColumnRenamed("Source","source").withColumnRenamed("id","trans_file_id")\
                .withColumnRenamed("Booking_channel","booking_channel").withColumnRenamed("Ticket_No","ticket_number").withColumnRenamed("PNR","pnr").withColumnRenamed("Issue_date","issue_date").withColumnRenamed("Tour_code","tour_code")\
                .withColumnRenamed("Passenger_name","passenger_name").withColumnRenamed("Passenger_Count","passenger_count").withColumnRenamed("Coupon_used","coupon_used").withColumnRenamed("Country_code","country_code")\
                .withColumnRenamed("Legs","legs").withColumnRenamed("Original_Fare","original_fare").withColumnRenamed("Orginal_currency","original_currency").withColumnRenamed("exch_rate","exchange_rate").withColumnRenamed("Fare_amt","fare_amount")\
                .withColumnRenamed("Commission_Amount","commission_amount").withColumnRenamed("Tax_Amt","tax_amount").withColumnRenamed("Total_amt","total_amount").withColumnRenamed("curr_code","currency_code").withColumnRenamed("FOP","payment")\
                .withColumnRenamed("Tax","tax") .withColumnRenamed("Commission","commission").withColumnRenamed("Fare_Cons","fare_construction").withColumnRenamed("file_path","refund_legs")


In [18]:
Bos_df_write_dyn=DynamicFrame.fromDF(Bos_df_write,glueContext,'Bos_df_write_dyn')

df3=glueContext.write_dynamic_frame_from_options(
    frame=Bos_df_write_dyn,
    connection_type="postgresql",
    connection_options=my_conn_options2,
    transformation_ctx="dynamic_frame"
)
#df3.show()

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/opt/amazon/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/opt/amazon/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/amazon/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 211, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/opt/amazon/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 132, in dump_stream
    for obj in iterator:
  File "/opt/amazon/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 200, in _batched
    for item in iterator:
  File "/opt/amazon/spark/python/lib/pyspark.zip/pyspark/worker.py", line 450, in mapper
    result = tuple(f(*[a[o] for o in arg_offsets]) for (arg_offsets, f) in udfs)
  File "/opt/amazon/spark/python/lib/pyspark.z

In [1]:
# # mini batch test
# from pyspark.sql import SQLContext
# sqlContest = SQLContext(sc)

# i = 0
# k = 19000
# j = 100

# while (i+j < k):
#     print(i)

#     try:
#         pdk = Bos_df_write.toPandas()
#         pd1 = pdk.loc[i:i+j][:].fillna(0)
#         spark_df = sqlContest.createDataFrame(pd1)
        
#         Bos_df_write_dyn1=DynamicFrame.fromDF(spark_df,glueContext,'Bos_df_write_dyn')

#         df3=glueContext.write_dynamic_frame_from_options(
#             frame=Bos_df_write_dyn1,
#             connection_type="postgresql",
#             connection_options=my_conn_options2,
#             transformation_ctx="dynamic_frame"
#         )
#     except Exception as e:
#         print(e)
#     finally:
#         i = i + j

    



    
    

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.37.3 
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::251954244960:role/aws-glue-role
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: 94e43814-4481-46a2-bd42-9a375b67a951
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.3
--enable-glue-datacatalog true
Waiting for session 94e43814-4481-46a2-bd42-9a375b67a951 to get into ready status...
Session 94e43814-4481-46a2-bd42-9a375b67a951 has been created.

